This is the ClassifySorB code written in python - both as a starting point for me and to examine the difference in Python and Julia.

In short, this code lacks a lot of features.

In [1]:
import pandas
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [2]:
df = pandas.read_csv("dataset.csv")

model = nn.Sequential(
    nn.Linear(3, 16),
    nn.LeakyReLU(),
    nn.BatchNorm1d(16),
    nn.Dropout(0.1),
    nn.Linear(16, 32),
    nn.Tanh(),
    nn.Dropout(0.2),
    nn.Linear(32, 12),
    nn.LeakyReLU(),
    nn.BatchNorm1d(12),
    nn.Dropout(0.3),
    nn.Linear(12, 3),
    nn.Sigmoid(),
    nn.BatchNorm1d(3),
    nn.Linear(3, 1)
).cuda()


In [3]:
losschecker = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

le = LabelEncoder()
df['class'] = le.fit_transform(df['class'])

X = df.drop('class', axis=1).values
y = df['class'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#turning the data into float32 tensors! Loss of accuracy for speed
X_train = torch.tensor(X_train, dtype=torch.float).cuda()
y_train = torch.tensor(y_train, dtype=torch.float).cuda()
X_test = torch.tensor(X_test, dtype=torch.float).cuda()
y_test = torch.tensor(y_test, dtype=torch.float).cuda()

#the loader and the number of epochs also had to be sacrificed for speed
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=128, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=1024, shuffle=True)

In [4]:
for epoch in tqdm(range(100)):
    
    for inputs, sol in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        sol = sol.view(-1, 1)

        loss = losschecker(outputs, sol)
        loss.backward()
        
        optimizer.step()

100%|██████████| 100/100 [04:09<00:00,  2.50s/it]


In [6]:
model.eval()

with torch.no_grad():
    correct = 0
    total = 0

    for inputs, sol in test_loader:
        
        outputs = model(inputs)
        sol = sol.view(-1, 1)
        predicted = torch.round(torch.sigmoid(outputs))

        total += sol.size(0)
        correct += (predicted == sol).sum().item()

print('Accuracy on test data: %d %%' % (100 * correct / total))

Accuracy on test data: 91 %
